In [79]:
from skimage.io import imread_collection , concatenate_images
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.utils import shuffle 
import random
import numpy as np
import matplotlib.pyplot as plt
import pywt as ptw
from sklearn.metrics import confusion_matrix
import copy as cp
from skimage.io.manage_plugins import _inject_imread_collection_if_needed




In [80]:
def draw(image):
    plt.imshow(image)
    plt.show()
    return

def hwavelet(images, n):
    cA = images
    for i in range(n):
        cA, (cH, cV, cD) = ptw.dwt2(cA, 'haar')
    return cA

def normalizar(dataset):
    aux = []
    for i in range(len(dataset)):
        aux = aux + [dataset[i].reshape(24*24).tolist()]
    return aux

def load(emotion, emotion_arr):
    aux = imread_collection(emotion_arr)
    aux_lb = [emotion] * len(aux)
    for i in range(len(aux)):
        all_images.append(aux[i])
        all_labels.append(aux_lb[i])



In [81]:
#paths
anger = 'images/anger/*.png'
contempt = 'images/contempt/*.png'
disgust = 'images/disgust/*.png'
fear = 'images/fear/*.png'
happy = 'images/happy/*.png'
sadness = 'images/sadness/*.png'
surprise = 'images/surprise/*.png'

#input y output
all_images = []
all_labels = []

#load en input y output
load('ANGER', anger)
load('CONTEMPT', contempt)
load('DISGUST', disgust)
load('FEAR', fear)
load('HAPPY', happy)
load('SADNESS', sadness)
load('SURPRISE', surprise)

#procesamiento
all_images_processed = hwavelet(all_images,1)
all_images_processed_normalized = normalizar(all_images_processed)

#print(len(all_images))
#print(len(all_labels))


In [82]:
def bootstrap(k,n):
    tam = len(all_images_processed) 
    t = tam
    ps = []
    result = []
    error = 0
    errores = []
    varianza = 0
    for i in range(tam):
        ps.append((all_images_processed[i], all_images_processed_normalized[i], all_labels[i]))
    ds = shuffle(ps)
    total_aciertos = 0
    total_samples = n
    if n==0:
        total_samples = 0
    
    for i in range(k):
        if n==0:
            n = random.randint(0,tam)
            total_samples += n
        idxs = random.sample(range(0,tam), n)
        
        train_i = []
        train_l = []
        test_i = []
        test_l = []

        for i in range(tam):
            if i in idxs:
                test_i.append(ds[i][1])
                test_l.append(ds[i][2])
            else:
                train_i.append(ds[i][1])
                train_l.append(ds[i][2])


        knn = KNeighborsClassifier(n_neighbors=1)

        knn.fit(train_i, train_l)

        aciertos = 0
        fallas = 0
        resultados = []
        for i in range(len(test_i)):
            predict = knn.predict([test_i[i]])
            resultados.append(predict)
            if predict == test_l[i]:
                aciertos += 1
            else:
                fallas += 1

        result.append(confusion_matrix(test_l, resultados, labels=["ANGER", "CONTEMPT", "DISGUST", "FEAR", "HAPPY", "SADNESS", "SURPRISE"]))
        # Error
        error_i = (fallas / (aciertos + fallas))
        error += error_i
        errores.append(error_i)

       # for i in range(len(test_l)): print(test_l[i],resultados[i])

        i += 1
    # Varianza
    error_promedio = error/k
    for error_i in errores:
        varianza += (error_i - error_promedio)**2
    varianza /= (k - 1)

    print('El error es de:', str(round(error_promedio * 100,2))+'%')
    print('La varianza es de:', varianza)

    return result
            #draw(test_im[i])
            #resultados.append('predict: '+ str(predict)+ ', real: ',str(test_l[i]))
            #if(i%3==0): print('predict: '+ str(predict)+ ', real: ',str(test_l[i]))
        

def kfold(k):
    tam = len(all_images_processed) 
    t = tam
    '''
    if tam % k != 0:
        print('choose another n')
        return
    '''
    diff = int(t/k)
    result = []
    ps = []

    for i in range(tam):
        ps.append((all_images_processed[i], all_images_processed_normalized[i], all_labels[i]))

    ds = shuffle(ps)

    #print(len(ds[1]))
    
    a = 0
    total_aciertos = 0

    error = 0
    errores = []
    varianza = 0
    for i in range(k):

        train_i = []
        train_l = []
        
        test_i = []
        test_l = []
        
        for i in range(t):
            
            if i in range(a,a+diff):
                test_i.append(ds[i][1])
                test_l.append(ds[i][2])
                
            else:
                train_i.append(ds[i][1])
                train_l.append(ds[i][2])
                
                
        a += diff

        knn = KNeighborsClassifier(n_neighbors=1)

        knn.fit(train_i, train_l)

        aciertos = 0
        fallas = 0
        resultados = []
        for i in range(len(test_i)):
            predict = knn.predict([test_i[i]])
            resultados.append(predict)
            if predict == test_l[i]:
                aciertos += 1
            else:
                fallas += 1
            #draw(test_im[i])
            #resultados.append('predict: '+ str(predict)+ ', real: ',str(test_l[i]))
            #if(i%3==0): print('predict: '+ str(predict)+ ', real: ',str(test_l[i]))
        result.append(confusion_matrix(test_l, resultados, labels=["ANGER", "CONTEMPT", "DISGUST", "FEAR", "HAPPY", "SADNESS", "SURPRISE"]))
        error_i = (fallas / (fallas+aciertos))
        error += error_i
        errores.append(error_i)

    error_promedio = error/k
    for i in errores:
        varianza += (i - error_promedio)**2
    varianza /= (k-1)

    print('El error es de:', str(round(error_promedio * 100,2))+'%')
    print('La varianza es de:', varianza)

    return result
               

    


    


In [83]:

def test_kfold(k):
    xd =kfold(k)
    diag = 0
    rest = 0
    count = 0
    for m in xd:
        for i in range(7):
            for j in range(7):
                if(i == j):
                    diag += m[i][j]
                else:
                    rest += m[i][j]
        print('accuracy for test '+str(count)+' ',diag/(rest+diag)*100)
        count+=1



def test_bootstrap(bf, ts):
    xd1 = bootstrap(bf,ts)
    diag = 0
    rest = 0
    count = 0
    for m in xd1:
        for i in range(7):
            for j in range(7):
                if(i == j):
                    diag += m[i][j]
                else:
                    rest += m[i][j]
        print('accuracy for test '+str(count)+' ',diag/(rest+diag)*100)
        count += 1

In [84]:
bootstrap_folds = 6
test_size = 300 #if 0, test_size randomizes every fold, train = ds-test_size

test_bootstrap(bootstrap_folds, test_size) 


kfold_folds = 3
test_kfold(kfold_folds)

El error es de: 7.72%
La varianza es de: 0.00019296296296296295
accuracy for test 0  91.66666666666666
accuracy for test 1  91.33333333333333
accuracy for test 2  92.22222222222223
accuracy for test 3  91.83333333333333
accuracy for test 4  92.0
accuracy for test 5  92.27777777777779
El error es de: 13.35%
La varianza es de: 0.0002462693313631788
accuracy for test 0  85.3211009174312
accuracy for test 1  85.77981651376146
accuracy for test 2  86.64627930682977
